In [71]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [72]:
def convert_to_one_hot(label,depth=10):
    encoded=tf.one_hot(label,depth,axis=0)
    
    return encoded

In [119]:
def model_build(layers):
    bias=tf.keras.initializers.HeNormal(seed=1)
    units=[25,15,10]
    for i in range(layers):
        if i==0:
            model.add(tf.keras.layers.Dense(units[i],activation="relu",kernel_initializer="he_normal",use_bias=True,bias_initializer=bias,input_shape=(784,)))
        elif i>0 and i!=layers-1:
            model.add(tf.keras.layers.Dense(units[i],activation="relu",kernel_initializer="he_normal",use_bias=True,bias_initializer=bias))
        elif i==layers-1:
            model.add(tf.keras.layers.Dense(units[i],activation="softmax",kernel_initializer="he_normal",use_bias=True,bias_initializer=bias))
            

In [75]:
def plot(learning):
    #Plotting Loss vs epochs
    plt.plot(learning.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Cost")
    plt.title("Learning Curve")
    plt.show()
    
    #Plotting Accuracy vs epochs
    plt.plot(learning.history['accuracy'])
    plt.xlabel("Epochs")
    plt.ylabel("Cost")
    plt.title("Accuracy Improvement")
    plt.show()    

In [76]:
ds=pd.read_csv("train.csv")
pred=pd.read_csv("test.csv")

In [77]:
x=ds.drop('label',axis=1)
y=ds['label']

In [78]:
x_train=np.array(x/255) #normalization
y_train=(np.array(convert_to_one_hot(y))).reshape((42000,10)) #One hot labels

In [79]:
#Converting to float

x_train=x_train.astype('float32')
y_train=y_train.astype('float32')

In [120]:
model=tf.keras.Sequential()

In [121]:
model_build(3)

In [126]:
learning_rate=0.00001
num_epochs=500
batchSize=32
show_costs=2

optimizer=tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=['accuracy'])
learning=model.fit(x_train,y_train,epochs=num_epochs,batch_size=batchSize,verbose=show_costs)


Epoch 1/100
1313/1313 [==============================] - 4s 2ms/step - loss: 2.3996 - accuracy: 0.0499
Epoch 2/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3294 - accuracy: 0.0506
Epoch 3/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3121 - accuracy: 0.0645
Epoch 4/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3069 - accuracy: 0.0888
Epoch 5/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3046 - accuracy: 0.0983
Epoch 6/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3031 - accuracy: 0.1050
Epoch 7/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3017 - accuracy: 0.0967
Epoch 8/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.3006 - accuracy: 0.0988
Epoch 9/100
1313/1313 [==============================] - 3s 2ms/step - loss: 2.2994 - accuracy: 0.1024
Epoch 10/100
1313/1313 [==============================] - 3s 2ms/step - l